In [5]:
import pandas as pd
import datetime as dt
import hashlib
import mysql.connector

In [6]:
conn = mysql.connector.connect(host='localhost',
                                         database='rossmann',
                                         user='root',
                                         password='root')
cursor = conn.cursor(buffered=True)


In [7]:
query= """ insert into sat_day_report (DAY_HSH,SALES_COUNT,CUSTOMER_COUNT)
        select h.DAY_HSH,  SUM(SALES), SUM(CUSTOMERS) FROM sat_day_facts s inner join link_day l 
        on s.LINK_DAY_HSH=l.LINK_DAY_HSH inner join hub_day  h  on h.DAY_HSH=l.DAY_HSH
        group by h.DAY_HSH; """ 

try:
    cursor.execute(query)
except mysql.connector.IntegrityError as e:
        print("error code:",e)

In [8]:
pd.read_sql_query(" Select * FROM sat_day_report;", con=conn).head(10)

,DAY_HSH,SALES_COUNT,CUSTOMER_COUNT
0,00147b0df0502364b1c76eaf89b2d469,5878702,736906
1,001c70302cc8bc9964693d52fb3d5b0c,7738889,852829
2,00de651aae828626819b1e6d71ca46ad,6008503,722458
3,013310d4f9f0b3fbb454958e069e1d2e,7728892,820715
4,014ee167e3d9942f4d0dbbcb25251e0b,10357418,1109623
5,0150a35d4d40a493b79381c702989272,4534864,531392
6,01802aa4d499775a9d1f0374a78c0afa,-856,-856
7,018b9f8fe5d596feb839d0e15431ecf9,6829093,733408
8,018f091c1c4acf18aa4860c095796128,8444592,908176
9,01aeda08914fe2668f4f16ec4f76a1e0,8084449,870146


In [ ]:
select 
    t.storeid as storeid,
    t.artid as artid,
    t.totamount,
    t.hour,
    (
        select 1 + count(*)
        from t t1
        where 
            t1.storeid = t.storeid 
            and t1.artid = t.artid
            and t1.hour > t.hour

    ) as rn
from t
order by t.storeid, t.artid

WITH t AS
 (select h.STORE_HSH,  AVG(s.SALES) as SALES_AVG , AVG(s.CUSTOMERS) as CSTMR_AVG
 FROM sat_day_facts s inner join link_day l
  on s.LINK_DAY_HSH=l.LINK_DAY_HSH inner join hub_store  h  on h.STORE_HSH=l.STORE_HSH
  group by h.STORE_HSH)

  select t.STORE_HSH, NOW() as LOAD_DATETIME, NOW() as LOAD_END_DATETIME, t.SALES_AVG, t.CSTMR_AVG,
  (  select 1 + count(*)
      from t t1
      where  t1.SALES_AVG > t.SALES_AVG) as SALES_RANK;


In [10]:
query= """   insert into sat_store_biz (STORE_HSH, LOAD_DATETIME, LOAD_END_DATETIME,SALES_AVG, CSTMR_AVG, SALES_RANK)
              WITH t as  (select h.STORE_HSH,  AVG(s.SALES) as SALES_AVG , AVG(s.CUSTOMERS) as CSTMR_AVG 
              FROM sat_day_facts s inner join link_day l 
              on s.LINK_DAY_HSH=l.LINK_DAY_HSH inner join hub_store  h  on h.STORE_HSH=l.STORE_HSH 
                  group by h.STORE_HSH) 
           select t.STORE_HSH, NOW() as LOAD_DATETIME, NOW() as LOAD_END_DATETIME, t.SALES_AVG, t.CSTMR_AVG,
          (  select 1 + count(*)
            from t t1
            where  t1.SALES_AVG > t.SALES_AVG
           ) as SALES_RANK
           from t
        
            ; """ 

try:
    cursor.execute(query)
except mysql.connector.IntegrityError as e:
        print("error code:",e)

In [ ]:
cursor.close()
conn.commit()
conn.close()